<a href="https://colab.research.google.com/github/wingated/cs473/blob/main/labs/cs473_lab_week_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><p><b>After clicking the "Open in Colab" link, copy the notebook to your own Google Drive before getting started, or it will not save your work</b></p>

# BYU CS 473 Lab Week 6

## Introduction:

The concept of mutual information is powerful in theory, but in practice it can be difficult to estimate. In this lab, you will implement a conceptually simple way to estimate the mutual information between two random variables. It is known as the KSG estimator (from "Kraskov–Stögbauer–Grassberger").

You will then use your estimator as part of a feature selection algorithm.

---
## Grading standards   

Your notebook will be graded on the following:

* 60% Correct implementation of KSG estimator
* 20% Correct implementation of "all_mutual_inf"
* 20% Correct selection of most informative columns

---
## Description

For this lab, we will use this dataset [lab_week_6_data.csv](https://github.com/wingated/cs473/blob/main/labs/data/lab_week_6_data.csv).

Download the csv from the link above, then upload the csv to your current google colab runtime.

Then begin by loading and preparing the dataset:

In [2]:
import pandas as pd
import numpy as np
from scipy.special import digamma
from sklearn.neighbors import NearestNeighbors

df = pd.read_csv( "lab_week_6_data.csv" )

X = df[['x1','x2','x3','x4','x5']]
y = df['y']

X = np.array(X)
y = np.array(y)


The KSG estimator is an elegant idea that's fairly easy to implement. The idea is to use an adaptive grid, where the grid depends on the local density of the data.  This is accomplished by calculating the k'th nearest neighbor for each point, and then defining a box around each point that just barely extends to include the k'th nearest neighbor. Importantly, the size of the box is also used in the marginal entropy calculations

To get some intuition for the method, check out [this demonstration on Wolfram Alpha's site](https://demonstrations.wolfram.com/KraskovKSGEstimatorOfMutualInformation/).  Here is a screenshot of their UI:

![Wolfram UI](https://raw.githubusercontent.com/wingated/cs473/main/labs/images/lab_week_6_image1.png)

The algorithm is fairly straightforward.  We are given N samples from the joint distribution of p(X,Y). First, we calculate the *local mutual information* for each data point:

1. For each datapoint $d_j = (x_j,y_j)$
2. Find the k'th nearest neighbor (call this point $n_j$) -- you should use the infinity norm (max norm), and distance should be calculated using both coordinates (ie, don't just calculate distances using only the x or y coordinate)
3. Count the number of points in x whose 'x-distance' fall within the distance to the k'th nearest neighbor (calculated above) of $d_j$ (ie, ignore y); call this number $n_{x{_j}}$.  Do the same for the points in y (these are the shaded gray boxes in the image); call it $n_{y{_j}}$.

Then, we calculate the final mutual information by averaging over all of the datapoints and adding a few more digamma calls:

![final mutual information](https://raw.githubusercontent.com/wingated/cs473/main/labs/images/lab_week_6_image3.png)

where $\Psi$ is the digamma function

If you want more details on the implementation, see page 2 of [Estimating Mutual Information](https://arxiv.org/pdf/cond-mat/0305641)

In [3]:
def mutual_inf( X, Y, k ):
    # X and Y are lists of length n of samples from p(X,Y)
    #
    # this function returns a single scalar
    if X.ndim > 1 or Y.ndim > 1:
        raise ValueError("X and Y must be 1-dimensional arrays.")
    if X.shape[0] != Y.shape[0]:
        raise ValueError("X and Y must have the same number of samples.")

    n_samples = X.shape[0]
    xy = np.c_[X,Y]

    nn = NearestNeighbors(n_neighbors=k + 1, metric='chebyshev')
    nn.fit(xy)
    distances, _ = nn.kneighbors(xy)
    radii = distances[:,-1]

    n_x = np.zeros( n_samples )
    n_y = np.zeros( n_samples )

    for i in range( n_samples ):
        r_i = radii[i]
        # Count points whose x-distance is strictly less than the radius.
        # This count includes the point itself.
        n_x[i] = np.sum(np.abs(X - X[i]) < r_i)
        # Count points whose y-distance is strictly less than the radius.
        n_y[i] = np.sum(np.abs(Y - Y[i]) < r_i)

        avg_digamma = np.mean(digamma(n_x) + digamma(n_y))
        mi = digamma(k) - avg_digamma + digamma(n_samples)

    return mi

---
## Part 2:

Now that you can estimate mutual information between two variables, we're going to use your method in the context of a regression problem.

Given our X,y data from the beginning of the lab, run your function and calculate the mutual information between every column of X and y.  This should result in a list of scores, that might look something like this:

```python
array([1.2641203602210282, 0.07974959311825458, 0.03321433628330617, 0.02213361087954091, 0.0, 0.0, 0.0])
```

If you wrap this in a single function (call it "all_mutual_inf"), then you can use scikit learn's feature selection algorithms:

In [4]:
def all_mutual_inf( X, y ):
    # return a list of mutual informations
    # the list should be as long as the number of columns in X

    # your code here
    n_features = X.shape[1]
    mi_scores = np.zeros(n_features)

    for i in range(n_features):
        # Calculate MI for the i-th feature column and y
        mi_scores[i] = mutual_inf(X[:, i], y, k=3) # Using k=3 as a default

    return mi_scores

In [5]:
from sklearn.feature_selection import SelectKBest
all_scores = all_mutual_inf(X, y)

print("--- Mutual Information Scores ---")
for i, score in enumerate(all_scores):
    print(f"Feature 'x{i+1}': {score:.4f}")

selector = SelectKBest(all_mutual_inf, k=2)
X_new = selector.fit_transform(X, y)

# Get the indices of the selected features
selected_indices = selector.get_support(indices=True)
feature_names = np.array(['x1', 'x2', 'x3', 'x4', 'x5'])
selected_features = feature_names[selected_indices]



--- Mutual Information Scores ---
Feature 'x1': 1.5617
Feature 'x2': 0.2381
Feature 'x3': 0.1888
Feature 'x4': 0.1728
Feature 'x5': -0.0824

--- Feature Selection Results ---
Shape of original data: (100, 5)
Shape of data after feature selection: (100, 2)

The two most informative features are: x1, x2


So: which X features are most informative about y?

In [6]:
print(f"Shape of original data: {X.shape}")
print(f"Shape of data after feature selection: {X_new.shape}")
print(f"\nThe two most informative features are: {', '.join(selected_features)}")

Shape of original data: (100, 5)
Shape of data after feature selection: (100, 2)

The two most informative features are: x1, x2


---
## Hints

The following functions may be useful to you:

In [ ]:
scipy.special.digamma

For fun, you could also [read more about the KSG estimator](https://arxiv.org/pdf/1604.03006).